## Analysis of human genome paired end Illumina reads

## Pipeline for Read Mapping and Visualization

* **Create the BWA Index** for the human hg38, chr 1 reference genome
    
    * For a small genome < 2 Gbases, use the **is** argument.  
      For example, hg38 chromosome 1 is ~250,000,000 nucleotides long.

    * For a large genome > 2GBases, use **bwtsw** argument.  
      For example hg38 is ~3,000,000,000 nucleotides long.
    
    * Parameters  
      * **-a** algorithm type, either **is** or **bwtsw**

In [ ]:
#create index for chromosome 1
#time: up to four minutes
!bwa index -a is chr1.fa

#create index for the entire genome 
#uncomment only if instructed to, this will take a long time
#!bwa index -a bwtsw hg38.fa

#check that files were created
!ls -ltr chr1.*    

* **Rename data files** to make the names shorter

In [ ]:
#copy and rename paired end files
!cp 071112_HWI-EAS192_0003_FC13191_PE_L1_YHPE_PE1.fq.gz yanhuang_PE1.fq.gz
!cp 071112_HWI-EAS192_0003_FC13191_PE_L1_YHPE_PE2.fq.gz yanhuang_PE2.fq.gz

#check that files were created
!ls -ltr yanhuang*    

* **Use BWA aln to map the read data** to human chromosome 1  
    
    **Parameters** are as follows:
    * **-n** maximum number of differences allowed between the read and the mapped location, here, 4% of the read length (default is 0.04)
    * **-o** maximum number of gap starts allowed (default is 1)
    * **-e** maximum number of gap extensions allowed (default is -1 means single character gaps)
    * **-t** number of threads (perhaps set to number of processor cores)
    * **files**
         * **chr1.fa** – reference fasta file
         * **yanhuang_PE1.fq.gz** – read data file in gzipped fastq format
         * **outyh1.sai** – output file (sai means suffix array indices).

In [ ]:
#map first reads of pairs
#time: at least two minutes
!bwa aln -n 0.04 -o 2 -e -1 -t 1 chr1.fa yanhuang_PE1.fq.gz > outyh1.sai

#map second reads of pairs
time: at least two minutes
!bwa aln -n 0.04 -o 2 -e -1 -t 1 chr1.fa yanhuang_PE2.fq.gz > outyh2.sai

#check that files were created (*.sai)
!ls -ltr outyh*    

* **Produce the SAM file output**  
  For **single reads** (not paired end), use **bwa samse**  
  For **paired end reads**, use the **bwa sampe**

  **Parameters** are as follows:
  * **-n** maximum number of best alignments to report (default is 3)
  * **files**
      * **chr1.fa** – reference fasta file
      * **outyh1.sai, outyh2.sai** - ???
      * **yanhuang_PE1.fq.gz,yanhuang_PE2.fq.gz** – read data file in fastq format (Note this file could be compressed, for example data.fastq.gz)
      * **alignmentsyh.sam** – SAM output file (this name is arbitrary)
      * for sampe, use two pairs of input .sai and .fastq files.    

In [ ]:
#produce sam file output
!bwa sampe -n 3 chr1.fa outyh1.sai outyh2.sai yanhuang_PE1.fq.gz yanhuang_PE2.fq.gz> alignmentsyh.sam

#check that files were created 
!ls -ltr alignmentsyh.sam    

* **Convert SAM to BAM** using the samtools **view** operation. BAM is a compressed version of SAM.

    **Parameters** are as follows:
    * **-b** output is in .bam format
    * **S** input is in .sam format and the SAM file contains header lines which give the length of each chromosome as above, for example: @SQ SN:chr1 LN:249250621

    * **files** 
        * **alignmentsyh.sam** – input paired end SAM alignment file
        * **alignmentsyh.bam** - output paired end SAM alignment file



In [ ]:
# convert paired end read sam file to bam file
!samtools view -bS alignmentsyh.sam > alignmentsyh.bam

# check that new files were created
!ls -ltr alignmentsyh.bam

**THE NEXT OPERATION IS NECESSARY ONLY IF THE SAM FILE 
DID NOT CONTAIN HEADER LINES WITH THE LENGTH OF EACH CHROMOSOME**

* **Create index file** with lengths of chromosomes using samtools **faidx** operation  

* **Convert SAM to BAM** using the samtools **view** operation. BAM is a compressed version of SAM.

    **Parameters** are as follows:
    * **-b** output is in .bam format
    * **-t** use an index file for chromosome lengths, each line contains a reference name and the length of the reference

    * **files** 
        * **chr1.fa** - source genome fasta file
        * **alignmentsyh.sam** – input paired end SAM alignment file
        * **alignmentsyh.bam** - output paired end SAM alignment file

In [ ]:
#uncomment commands in this cell only if needed as stated above

# create index file named .fai  
#!samtools faidx chr1.fa

# *check that new file was created  
#!ls -ltr *.fai

# convert sam file to bam file  
#!samtools view -bt chr1.fa.fai alignments.sam > alignments.bam
 
# check that new files were created 
#!ls -ltr *.bam

* **Sort the BAM file** using the samtools **sort** function

    This command sorts the reads by chromosome and position.

    **Note that .bam will be appended to the end of the output file name. Do not put .bam at the end of the name**

In [ ]:
# sort the bam file
!samtools sort alignmentsyh.bam -o alignmentsyh.sorted 

# check that new file was created
!ls -ltr alignmentsyh.sorted.bam

* **Index the BAM file** using the samtools **index** function

    This command creates a .bai index file for fast lookup of the reads by chromosome and position.  
    
    **It produces a .bai index file that is used to quickly locate the reads, for example, by the IGV viewer**.

In [ ]:
#index the paired end read sorted bam file
!samtools index alignmentsyh.sorted.bam 

# check that new files were created
!ls -ltr alignmentsyh.sorted.bam.bai

* **Check the number of reads mapped** using the samtools **idxstats** function

    **Output is chromosome name, length, number of reads mapped, and number of reads not mapped.**

In [ ]:
#print headers for the ouput
!echo "\nPaired end reads"
!echo "chr     length          mapped  not_mapped"

#check reads mapped in the paired end read sorted bam file
!samtools idxstats alignmentsyh.sorted.bam

**Optional** if bam files are produced separately
* **Merge bam files** using samtools **merge** function 
    * **files**
        * **alignments.together.bam** - output alignment file
        * **alignments1.sorted.bam, alignments2.sorted.bam** - input alignment files

In [ ]:
#optional, only uncomment if there are multiple bam files

# merge alignment files
#!samtools merge alignments.together.bam

# check that new file was created
#!ls -ltr *.together.*

* **Extract** alignments for a particular region from BAM/SAM using samtools **view** function

    Specified either as a **chromosome** or as a **chromosome and a range** 
    
    To use this **with IGV**, produce a **separate index file** using **samtools index** command


In [ ]:
# extract alignments for entire chromosome 1
!samtools view -b alignmentsyh.sorted.bam chr1 > alignmentsyh.sorted.chr1.bam
                        
#extract alignments for a range within chromosome 1    
!samtools view -b alignmentsyh.sorted.bam chr1:100000000-200000000 > alignmentsyh.sorted.chr1.100Mto200M.bam

# check that new files were created
!ls -ltr *.chr1*bam

#index the paired end chromosome 1 alignments bam file
!samtools index alignmentsyh.sorted.chr1.bam

#index the paired end range within chromosome 1 alignments bam file
!samtools index alignmentsyh.sorted.chr1.100Mto200M.bam

# check that new files were created
!ls -ltr *chr1*bai

* **View mapped reads in the Integrated Genomics Viewer (IGV)**

    * **Start IGV** by clicking on the webstart file or icon.   
    Use of IGV may require resetting security parameters for Java.  
    Shows mapped sequencing reads (and other data) along a reference genome. 
    
    * **Orientation:**
        1. **Load reference genome:** In menu box at upper left, select **hg38**  
        2. **Choose chromosome:** In second text box upper left choose **chr 1**
        3. **Load data:** in menu select File->Load from File
            * choose alignmentsyh.sorted.chr1.bam
        4. **Choose an interval**
            * click on chromosome image
            * use zoom slider bar on upper right
            * user Go text box in upper middle 
                * use chr1:117,147,212-117,170,177  
            * drag view left and right with mouse
            * Sequence is visible only at higher zoom
        5. **View refseq genes** in the bottom feature area  
            * click on refseq gene track to go to Genbank data
        6. **Add more features**  
            * use File>Load from server
            * Available Datasets>Annotations> Variation and Repeats->dbSNP 1.3.7
        7. **Get more information** 
            * Cursor hover over features
        8. **Options** using right click
            * Paired reads have same color
            * View as pairs shows pairs with line connecting them
            * Change read colors based on insert size and/or read orientation6. 